In [1]:
import pandas as pd
import numpy as np

from surprise import Reader
from surprise import Dataset
from surprise import SVD


In [2]:
train_data = pd.read_csv("../data/train-PDA2019.csv")
train_data.columns = ["userID", "itemID", "rating","timeStamp"]
train_data = train_data.sort_values("userID").reset_index(drop=True)
train_data = train_data.iloc[:,0:3]
train_data.head()

,userID,itemID,rating
0,1,919,4
1,1,608,4
2,1,2797,4
3,1,1545,4
4,1,594,4


In [3]:
test_data = pd.read_csv("../data/test-PDA2019.csv")
test_data.columns = ["userID", "recommended_itemID"]

In [4]:
content_df = pd.read_csv("../data/content-PDA2019.csv")
#content_df.head()

In [5]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(train_data[['userID', 'itemID','rating']], reader)

In [6]:
rating_df = data.build_full_trainset()

In [7]:
user_rating = rating_df.ur
item_rating = rating_df.ir
num_users = rating_df.n_users
num_items = rating_df.n_items
num_ratings = rating_df.n_ratings

mean_ = rating_df.global_mean
mean_ = np.round(mean_, decimals=2)

density_ = num_ratings/(num_users*num_items)
sparsity_ = 1 - density_


In [8]:
print("Users", num_users)
print("items", num_items)
print("ratings ", num_ratings)
print("Mean rating", mean_)
print("density " ,density_ *100, "%")
print("sparsity ", sparsity_ *100 ,"%")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
#Best params for best fcp ranking gives better results than the params for best rmse
recommender = SVD(n_epochs= 10, lr_all= 0.001, reg_all= 0.5)

In [45]:
'''
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import accuracy

kf = KFold(n_splits=4)
'''

'\nfrom surprise.model_selection import cross_validate\nfrom surprise.model_selection import KFold\nfrom surprise import accuracy\n\nkf = KFold(n_splits=4)\n'

In [46]:
'''
fold_n = 1

for trainset, testset in kf.split(data):
    recommender.fit(trainset)
    predictions = recommender.test(testset)
    
    rmse_ = accuracy.rmse(predictions, verbose=False)
    mae_ = accuracy.mae(predictions, verbose=False)
    rank_qualit_ = accuracy.fcp(predictions, verbose=False)
    
    print('--- Fold: ', fold_n, '----')
    print('Pred accuracy (MAE): {:.2f}'.format(mae_), ','
          'Pred accuracy (RMSE): {:.2f}'.format(rmse_), ','
          'Pred accuracy (FCP): {:.2f}'.format(rank_qualit_)) 
    
    fold_n +=1
'''

"\nfold_n = 1\n\nfor trainset, testset in kf.split(data):\n    recommender.fit(trainset)\n    predictions = recommender.test(testset)\n    \n    rmse_ = accuracy.rmse(predictions, verbose=False)\n    mae_ = accuracy.mae(predictions, verbose=False)\n    rank_qualit_ = accuracy.fcp(predictions, verbose=False)\n    \n    print('--- Fold: ', fold_n, '----')\n    print('Pred accuracy (MAE): {:.2f}'.format(mae_), ','\n          'Pred accuracy (RMSE): {:.2f}'.format(rmse_), ','\n          'Pred accuracy (FCP): {:.2f}'.format(rank_qualit_)) \n    \n    fold_n +=1\n"

In [47]:
recommender.fit(rating_df)


In [48]:
# Get a list of all individual items from content.csv
#items_all = content_df.iloc[:,0].as_matrix()
#len(items_all)

In [49]:
items_train = train_data["itemID"]
items = items_train.sort_values()
items = items.as_matrix(items)
items_set = set(items)
len(items_set)

c:\users\katha\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


845

In [50]:
#get a list of all users to iterate over
users_all = test_data["userID"].as_matrix()
len(users_all)

c:\users\katha\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


1992

In [51]:
predictions = []
item_pred = []
i = 0
for user in users_all:
    for item in items_set:
        predictions += recommender.predict(user,item)
    i += 1

In [52]:
# 0 is the user
# 1 is the item id
# 3 is the prediction
#predictions

In [53]:
ind_pred = []
for i in range(0, len(predictions), 5):
    ind_pred.append(predictions[i:i + 5])
        

In [54]:
columns = ["userID", "itemID", "none", "predEst", "impossible"]
df = pd.DataFrame(ind_pred , columns=columns)

In [55]:
df = df.sort_values(by=['userID','predEst'], ascending=False)
df

,userID,itemID,none,predEst,impossible
1682527,12073,318,None,4.211248,{'was_impossible': False}
1682749,12073,858,None,4.196685,{'was_impossible': False}
1682786,12073,922,None,4.167136,{'was_impossible': False}
1682893,12073,1148,None,4.165047,{'was_impossible': False}
1682764,12073,904,None,4.140810,{'was_impossible': False}
1682771,12073,912,None,4.132076,{'was_impossible': False}
1683218,12073,1207,None,4.130620,{'was_impossible': False}
1682499,12073,260,None,4.126119,{'was_impossible': False}
1682910,12073,1198,None,4.126111,{'was_impossible': False}
1682927,12073,1212,None,4.106118,{'was_impossible': False}


In [56]:
df = df.iloc[:,0:4]
len(df["userID"])
df

,userID,itemID,none,predEst
1682527,12073,318,None,4.211248
1682749,12073,858,None,4.196685
1682786,12073,922,None,4.167136
1682893,12073,1148,None,4.165047
1682764,12073,904,None,4.140810
1682771,12073,912,None,4.132076
1683218,12073,1207,None,4.130620
1682499,12073,260,None,4.126119
1682910,12073,1198,None,4.126111
1682927,12073,1212,None,4.106118


In [57]:
top10 = []
for i in range(0, len(df["userID"]), len(items_set)):
    top10.append(df[i:i+10])


In [58]:
# User ID and 10 Item IDs
cols = ["userID", "recommended_itemIDs"]
top10_df = pd.DataFrame(columns = cols)

top10_df.iloc[:,0:2]

,userID,recommended_itemIDs


In [59]:
#---------TESTING
value = top10[0].iloc[2]['userID']

In [60]:
items_string = ""
items_str_list = []
users_list = []
for elem in top10:
    listtest = elem["itemID"].tolist()
    stringtest = str(listtest).strip('[]').replace(",", " ")
    items_str_list.append(stringtest)
    users_list.append(elem.iloc[2]['userID'])


In [61]:
top10_df["recommended_itemIDs"] = items_str_list
top10_df["userID"] = users_list

In [63]:
top10_df = top10_df.sort_values("userID")


In [64]:
top10_df.to_csv("../data/competition15.csv", index=False)